This is a notebook for:
1. Read airports table
2. Connect to Rapid API
3. Collect flights data for airports
4. Sava data into DataFrame
5. Update flights table with collected data

The flights table should contain:
* flights_id
* flights_number
* departure_icao
* departure_city
* departure_time_local
* arrival_icao
* arrival_time_local
* arrival_terminal
* airline
* data_retrieved_on

### 1. Import of the needed modules and stuff:

In [9]:
import pandas as pd
import sqlalchemy
import keys
import requests
import datetime

In [10]:
# Safely stored API:
rapid_API = keys.rapid_API

### 2. From "gans":

In [11]:
host = '127.0.0.1'
schema = 'gans'
user = 'root'
password = keys.mysql_PW
port = 3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [12]:
# Airports:
airports_df = pd.read_sql('airports',con=con)
airports_df

,city_id,icao


### 3. From rapidapi website:

In [13]:
url = "https://aerodatabox.p.rapidapi.com/flights/airports/icao/EHAM/2022-10-04T20:00/2022-10-05T08:00"

querystring = {"withLeg":"true","direction":"Arrival","withCancelled":"false","withCodeshared":"true","withCargo":"true","withPrivate":"false","withLocation":"false"}

headers = {
	"X-RapidAPI-Key": rapid_API,
	"X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)

{"arrivals":[{"departure":{"airport":{"icao":"EPKK","iata":"KRK","name":"Kraków"},"quality":[]},"arrival":{"scheduledTimeLocal":"2022-10-04 19:55+02:00","actualTimeLocal":"2022-10-04 20:15+02:00","runwayTimeLocal":"2022-10-04 20:15+02:00","scheduledTimeUtc":"2022-10-04 17:55Z","actualTimeUtc":"2022-10-04 18:15Z","runwayTimeUtc":"2022-10-04 18:15Z","terminal":"2","baggageBelt":"11","quality":["Basic","Live"]},"number":"AF 8288","status":"Arrived","codeshareStatus":"IsCodeshared","isCargo":false,"aircraft":{"model":"Embraer 195-E2"},"airline":{"name":"Air France"}},{"departure":{"airport":{"icao":"EKBI","iata":"BLL","name":"Billund"},"quality":[]},"arrival":{"scheduledTimeLocal":"2022-10-04 19:50+02:00","actualTimeLocal":"2022-10-04 20:57+02:00","runwayTimeLocal":"2022-10-04 20:57+02:00","scheduledTimeUtc":"2022-10-04 17:50Z","actualTimeUtc":"2022-10-04 18:57Z","runwayTimeUtc":"2022-10-04 18:57Z","terminal":"2","baggageBelt":"12","quality":["Basic","Live"]},"number":"EY 5814","status":"A

### 4. Setting up:

In [14]:
# Producing timestamp:
today = datetime.date.today()
tomorrow = str(today + datetime.timedelta(days=1))
# The needed times since API provides 12h:    
times = [["00:00","11:59"],["12:00","23:59"]]

In [15]:
icao_list = airports_df['icao'].to_list()
icao_list[1:2]

flights_dict = {'ICAO':[],'departure_airport':[],'local_time':[]}
icao_list = airports_df['icao'].to_list()

for time in times:   
    for icao in icao_list[1:2]:
        url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{tomorrow}T{time[0]}/{tomorrow}T{time[1]}"
        response = requests.request("GET", url, headers=headers, params=querystring)
        print(response)
        flight_json = response.json()        
        for flight in flight_json['arrivals']:
            flights_dict['ICAO'].append(icao)
            try:
                flights_dict['departure_airport'].append(flight['departure']['airport']['icao'])
            except:
                flights_dict['departure_airport'].append('unknown')
            try:
                flights_dict['local_time'].append(flight['arrival']['scheduledTimeLocal'])
            except:
                flights_dict['local_time'].append(pd.NaT)

In [16]:
flights_df = pd.DataFrame(flights_dict)
flights_df['local_time'] = pd.to_datetime(flights_df['local_time'])
flights_df.to_sql('flight',con=con,if_exists='append',index=False)

0